# Audio Recording with PyAudio

In [6]:
import pyaudio

import wave

import keyboard

import threading

In [7]:
#Params
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS_MAX = 10  # Set the maximum recording duration in seconds

### Instantiate PyAudio Object

In [8]:
p = pyaudio.PyAudio()

In [10]:
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print(f"Recording... (Press 'Control + C' to stop or maximum duration is {RECORD_SECONDS_MAX} seconds)")


frames = []

def stop_recording():
    global RECORD_SECONDS
    RECORD_SECONDS = RECORD_SECONDS_MAX


# Start a timer to stop recording after RECORD_SECONDS_MAX seconds
RECORD_SECONDS_MAX = 10
RECORD_SECONDS = 0
timer = threading.Timer(RECORD_SECONDS_MAX, stop_recording)
timer.start()


try:
    while RECORD_SECONDS < RECORD_SECONDS_MAX:
        data = stream.read(CHUNK)
        frames.append(data)
except KeyboardInterrupt:
    # Ctrl+C was pressed, stop recording
    print("Finished recording.")
    pass

print("Finished recording.")


stream.stop_stream()
stream.close()
p.terminate()





WAVE_OUTPUT_FILENAME = "data/raw/test2.wav"
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

Recording... (Press 'Control + C' to stop or maximum duration is 10 seconds)
Finished recording.


# Audio Transcription with Whisper